In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import sys
sys.path.append('..')

import bayesbridge.reg_coef_sampler.hamiltonian_monte_carlo as hmc
from bivariate_skewnormal import BivariateSkewNormal

## Run HMC on a bivariate Gaussian target.

In [ ]:
sigma = np.array([1.0, 2.0])
rho = 0.9

Sigma = np.array([
    [sigma[0] ** 2, rho * np.prod(sigma)],
    [rho * np.prod(sigma), sigma[1] ** 2]
])
Phi = np.linalg.inv(Sigma)

def bivariate_gaussian_target(x):
    logp = - np.inner(x, Phi.dot(x)) / 2
    grad = - Phi.dot(x)
    return logp, grad

def compute_marginal_pdf(x, axis):
    pdf = 1 / np.sqrt(2 * np.pi) / sigma[axis] \
        * np.exp(- x ** 2 / 2 / sigma[axis] ** 2)
    return pdf

In [ ]:
eigvals, _ = np.linalg.eig(Phi)
pc_scale = 1 / np.sqrt(eigvals)

# Choose near-optimal stepsize and integration time.
dt = np.array([.7, .8]) * 2. * np.min(pc_scale)
n_step = np.ceil(
    np.array([.8, 1.]) * np.max(pc_scale) / np.max(dt)
)

n_burnin = 10
n_sample = 10 ** 5

theta0 = np.zeros(2)
f = bivariate_gaussian_target
samples, logp_samples, accept_prob, time_elapsed \
    = hmc.generate_samples(f, theta0, dt, n_step, n_burnin, n_sample)

In [ ]:
plt.figure(figsize=(14, 10))
plt.rcParams['font.size'] = 20

bins = [
    sigma[0] * np.linspace(-4, 4, 51),
    sigma[1] * np.linspace(-4, 4, 51)
]
marginal_pdf = [
    compute_marginal_pdf(bins[axis], axis) for axis in [0, 1]
]

plt.subplot(2, 2, 1)
plt.hist2d(samples[0, :], samples[1, :],
           bins=bins, normed=1, cmap='inferno')
plt.ylabel(r'$\theta_2')
plt.xlabel(r'$\theta_1$')
plt.colorbar(ticks=[])

for axis in [0, 1]:
    plt.subplot(2, 2, 3 - axis)
    plt.hist(samples[axis, :], bins=bins[axis], normed=1)
    plt.plot(bins[axis], marginal_pdf[axis])
    plt.xlabel(r'$\theta_{:d}$'.format(axis + 1))

plt.tight_layout()
plt.show()

## Run HMC on a rotated 2-d skew normal.
Gaussians have many special features that makes them particularly simple for HMC, so additional sanity check with a non-Gaussian target won't hurt.

In [ ]:
skewnorm = BivariateSkewNormal(
    shape=np.array([2, 4]), 
    rotation=np.array([
        [1, 1],
        [1, -1]
    ]) / np.sqrt(2)
)

f = lambda x: (
    skewnorm.compute_logp(x), 
    skewnorm.compute_gradient(x)
)

from tests.test_gradient import numerical_grad_is_close
x = np.random.randn(2)
assert numerical_grad_is_close(f, x)

In [ ]:
dt = np.array([.25, .3])
n_step = np.array([3, 6])
n_burnin = 10
n_sample = 10 ** 5

theta0 = np.zeros(2)
samples, logp_samples, accept_prob, time_elapsed \
    = hmc.generate_samples(f, theta0, dt, n_step, n_burnin, n_sample)

In [ ]:
plt.figure(figsize=(14, 10))
plt.rcParams['font.size'] = 20

bins = [
    np.linspace(-2, 4, 51),
    np.linspace(-3, 3, 51)
]
marginal_pdf = skewnorm.compute_marginal_pdf(*bins)

plt.subplot(2, 2, 1)
plt.hist2d(samples[0, :], samples[1, :],
           bins=bins, normed=1, cmap='inferno')
plt.ylabel(r'$\log_{10}(N)$')
plt.xlabel(r'$\log(p / (1 - p))$')
plt.colorbar(ticks=[])

for axis in [0, 1]:
    plt.subplot(2, 2, 3 - axis)
    plt.hist(samples[axis, :], bins=bins[axis], normed=1)
    plt.plot(bins[axis], marginal_pdf[axis])
    plt.xlabel(r'$\theta_{:d}$'.format(axis + 1))

plt.tight_layout()
plt.show()